# **Tutorial** - Dynamic Topic Modeling with Trump's Tweets
(last updated 11-09-2022)

In this tutorial we will be using Dynamic Topic Modeling with BERTopic to visualize how topics in Trump's Tweets have evolved over time. These topics will be visualized and thoroughly explored.

## Dynamic Topic Models
Dynamic topic models can be used to analyze the evolution of topics of a collection of documents over time.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this tutorial, we will be needing to extract all Trump's Tweet from his @realDonalTrump account. We will be removing all retweet and focus on his original tweets.

Moreover, since we are looking at his tweets over time, we will be saving all timestamps related to his tweets.

In [ ]:
import re
import pandas as pd
from datetime import datetime

# Load data
trump = pd.read_csv('https://drive.google.com/uc?export=download&id=1xRKHaP-QwACMydlDnyFPEaFdtskJuBa6')

# Filter
trump.text = trump.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
trump.text = trump.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
trump.text = trump.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
trump = trump.loc[(trump.isRetweet == "f") & (trump.text != ""), :]
timestamps = trump.date.to_list()
tweets = trump.text.to_list()

In [ ]:
tweets[0]

'republicans and democrats have both created our economic problems'

# **Dynamic Topic Modeling**


## Basic Topic Model
To perform Dynamic Topic Modeling with BERTopic we will first need to create a basic topic model using all tweets. The temporal aspect will be ignored as we are, for now, only interested in the topics that reside in those tweets.

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(min_topic_size=35, verbose=True)
topics, _ = topic_model.fit_transform(tweets)

2024-03-17 21:26:12,692 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

2024-03-17 21:26:40,801 - BERTopic - Embedding - Completed ✓
2024-03-17 21:26:40,802 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 21:27:54,710 - BERTopic - Dimensionality - Completed ✓
2024-03-17 21:27:54,714 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 21:28:00,334 - BERTopic - Cluster - Completed ✓
2024-03-17 21:28:00,356 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-03-17 21:28:02,392 - BERTopic - Representation - Completed ✓


We can then extract most frequent topics:

In [ ]:
freq = topic_model.get_topic_info(); freq.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,15101,-1_the_to_of_is,"[the, to, of, is, and, that, he, on, you, it]",[the great people know after years of this pre...
1,0,3029,0_run_president_trump_please,"[run, president, trump, please, mr, donald, yo...",[gcjaramillo please run for president this cou...
2,1,2028,1_crowd_carolina_thank_join,"[crowd, carolina, thank, join, rally, iowa, ha...",[on my way to south carolina big crowd look fo...
3,2,1077,2_golf_course_doral_scotland,"[golf, course, doral, scotland, club, turnberr...",[receiving the trophy for best new golf course...
4,3,1037,3_border_wall_mexico_immigration,"[border, wall, mexico, immigration, security, ...",[our southern border is under siege congress m...
5,4,878,4_hotel_tower_chicago_building,"[hotel, tower, chicago, building, sign, trump,...",[check out trump international hotel tower new...
6,5,830,5_china_trade_tariffs_chinese,"[china, trade, tariffs, chinese, us, farmers, ...",[china buys much less from us than we buy from...
7,6,643,6_obamacare_healthcare_repeal_website,"[obamacare, healthcare, repeal, website, premi...",[it was a great day for the united states of a...
8,7,605,7_amp_it_to_you,"[amp, it, to, you, we, are, for, great, will, be]",[good practice as you saw last night they were...
9,8,565,8_media_fake_news_failing,"[media, fake, news, failing, story, dishonest,...",[the fake news media failing is not my enemy i...


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(7)

[('amp', 0.04356275664168553),
 ('it', 0.006417360273265924),
 ('to', 0.006195442681642055),
 ('you', 0.006150167897217806),
 ('we', 0.006072358225224138),
 ('are', 0.005690168581192336),
 ('for', 0.005083848249442357),
 ('great', 0.005066059670336437),
 ('will', 0.0049355108889816085),
 ('be', 0.004774549861151521)]

We can visualize the basic topics that were created with the Intertopic Distance Map. This allows us to judge visually whether the basic topics are sufficient before proceeding to creating the topics over time.

In [ ]:
fig = topic_model.visualize_topics(); fig

## Topics over Time
Before we start with the Dynamic Topic Modeling step, it is important that you are satisfied with the topics that were created previously. We are going to be using those specific topics as a base for Dynamic Topic Modeling.

Thus, this step will essentially show you how the topics that were defined previously have evolved over time.

There are a few important parameters that you should take note of, namely:

* `docs`
  * These are the tweets that we are using
* `timestamps`
  * The timestamp of each tweet/document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20.


In [ ]:
topics_over_time = topic_model.topics_over_time(docs=tweets,
                                                timestamps=timestamps,
                                                global_tuning=True,
                                                evolution_tuning=True,
                                                nr_bins=20)

20it [00:08,  2.37it/s]


## Visualize Topics over Time
After having created our `topics_over_time`, we will have to visualize those topics as accessing them becomes a bit more difficult with the added temporal dimension.

To do so, we are going to visualize the distribution of topics over time based on their frequency. Doing so allows us to see how the topics have evolved over time. Make sure to hover over any point to see how the topic representation at time *t* differs from the global topic representation.


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)